# Prepare ATMOS file for REF ET

### Import needed libraries

In [1]:
import pandas as pd
import numpy as np
import refet
import pytz

In [2]:
# where is the file coming from? A CSI Logger? ZentraCloud? Decagon Webviewer? ZentraUtility? ECH2O Utility?
# some other custom format? We need to have sensible defaults to make this as easy as possible.
#filename = input("Em60 Data file: ")

dfp1 = pd.read_excel("/home/acampbell/Public/06-00173 24May18-1146.xlsx",
             header=[0,1,2], 
             #index_col=0,
             )
dfp2 = pd.read_excel("/home/acampbell/Public/06-00173 30May18-0956.xlsx",
             header=[0,1,2], 
             #index_col=0,
             )
df3 = pd.read_excel("/home/acampbell/Public/06-00173 01Jun18-1242.xlsx",
             header=[0,1,2],
             #index_col=0,
             )

df = dfp1.append(dfp2.append(df3))

### Cut the needed columns

In [3]:
result = df['Port 1']['ATMOS 41 All-in-one Weather Station'][
    [
        'W/m² Solar Radiation',
        'm/s Wind Speed',
        'RH Relative Humidity',
        'mm Precipitation',
        '°C Air Temperature',
    ]
]

### Bin data to be hourly

In [4]:
hourly = result.resample('H').mean()
hourly['mm Precipitation'] = result['mm Precipitation'].resample('H').sum()

In [5]:
#result['Time'] = result.index.time

#result['Date'] = result.index.date

#### REF ET program requires times to be in hhmm format

#result['Time']=result['Time'].apply(str).apply(lambda x: x[0:2]+x[3:5])

#### REF ET program requires dates to be in yyyymmdd format

#result['Date']=result['Date'].apply(str).apply(lambda x: x[0:4]+x[5:7]+x[8:10])

### Output the CSV file

#result.to_csv("Public/21may18 ATMOS.csv",sep=",",index=False,)

## Prepare data for use with refet python library

In [6]:
## ea -- VP (array) CONVERT FROM RH
# RH = vp/vps, where vp is vapor pressure, and vps is saturation vapor pressure.
# vps can be found using Teten's formula.
# P = 0.61078 exp(17.27*T/(T+237.3)) for T over 0 C
hourly['VP kPa'] = hourly['RH Relative Humidity'] * (
    0.61078 ** (
        17.27*hourly['°C Air Temperature']/(
            hourly['°C Air Temperature'] + 237.3)
    )
)
ea = hourly['VP kPa']

## rs -- Solar Radiation (array) UNIT MISMATCH, use input_units dict
rs = hourly['W/m² Solar Radiation']

## uz -- wind speed (array)
uz = hourly['m/s Wind Speed']

## zw -- wind speed height (float)
zw = 2 # 2m

## elev -- elevation (array)
hourly['elev'] = 1408
elev = hourly['elev'] # 1408m above sea level

## lat -- latitude in degrees (array)
hourly['lat'] = 40.246025
lat = hourly['lat']

## doy -- day of year (array)
# This can be calculated using Pandas.
doy = hourly.index.dayofyear

## tmean -- average temperature (array)
tmean = hourly['°C Air Temperature']

## lon -- longitude in degrees (array)
hourly['lon'] = 111.64134
lon = hourly['lon']

## time -- UTC hour at start of time period (array)
time = hourly.index.tz_localize("America/Denver").tz_convert(None).hour

## method -- 'refet' to follow RefET software
method = 'refet'

## input_units -- {'rs':'w/m2'} Everything else mathces the default units
input_units = {'rs':'w/m2'}


In [7]:
hourly['ETo mm/hr'] = refet.Hourly(
    tmean=tmean, ea=ea, rs=rs, uz=uz, zw=zw, elev=elev,
    lat=lat, lon=lon, doy=doy, time=time, method=method,
    input_units=input_units).eto()

/home/acampbell/.local/lib/python3.6/site-packages/refet/calcs.py:569: RuntimeWarning: invalid value encountered in minimum
  kd = np.minimum(-0.36 * kb + 0.35, 0.82 * kb + 0.18)
/home/acampbell/.local/lib/python3.6/site-packages/refet/calcs.py:653: RuntimeWarning: invalid value encountered in greater
  fcd[rso > 0] = 1.35 * np.clip(rs[rso > 0] / rso[rso > 0], 0.3, 1) - 0.35
/home/acampbell/.local/lib/python3.6/site-packages/refet/hourly.py:209: RuntimeWarning: invalid value encountered in less
  self.cd[self.rn < 0] = 0.96
/home/acampbell/.local/lib/python3.6/site-packages/refet/hourly.py:210: RuntimeWarning: invalid value encountered in less
  self.g_rn[self.rn < 0] = 0.5


In [8]:
hourly['ETo mm/hr']

2018-05-21 13:00:00    0.231157
2018-05-21 14:00:00    0.374053
2018-05-21 15:00:00    0.509361
2018-05-21 16:00:00    0.369926
2018-05-21 17:00:00    0.259247
2018-05-21 18:00:00    0.105401
2018-05-21 19:00:00    0.069564
2018-05-21 20:00:00    0.043330
2018-05-21 21:00:00    0.080827
2018-05-21 22:00:00    0.041810
2018-05-21 23:00:00    0.045603
2018-05-22 00:00:00    0.040416
2018-05-22 01:00:00    0.032979
2018-05-22 02:00:00    0.029081
2018-05-22 03:00:00    0.028652
2018-05-22 04:00:00    0.028693
2018-05-22 05:00:00    0.006047
2018-05-22 06:00:00    0.028396
2018-05-22 07:00:00    0.096510
2018-05-22 08:00:00    0.169598
2018-05-22 09:00:00    0.205775
2018-05-22 10:00:00    0.358132
2018-05-22 11:00:00    0.592278
2018-05-22 12:00:00    0.167790
2018-05-22 13:00:00    0.294221
2018-05-22 14:00:00    0.177883
2018-05-22 15:00:00    0.170776
2018-05-22 16:00:00    0.403053
2018-05-22 17:00:00    0.175729
2018-05-22 18:00:00    0.127562
                         ...   
2018-05-

# Sum ET output per day

In [9]:
daily_totals = hourly['ETo mm/hr']['2018-05-25':'2018-06-01'].resample('D').mean()*24
print(daily_totals,'\n',hourly['ETo mm/hr']['2018-05-25':'2018-06-01'].resample('D').sum())

2018-05-25    6.067844
2018-05-26    5.735536
2018-05-27    3.357657
2018-05-28    4.016418
2018-05-29    4.771153
2018-05-30    5.404688
2018-05-31    6.310179
2018-06-01    3.202352
Freq: D, Name: ETo mm/hr, dtype: float64 
 2018-05-25    6.067844
2018-05-26    5.735536
2018-05-27    3.357657
2018-05-28    4.016418
2018-05-29    4.771153
2018-05-30    5.404688
2018-05-31    6.310179
2018-06-01    1.734607
Freq: D, Name: ETo mm/hr, dtype: float64


In [10]:
daily_totals['2018-05-29':'2018-05-31'].mean()

5.4953400891898445